In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 1}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.49772417545318604],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.00031507934909313917,
  'n': 512,
  'skewness': None,
  'stddev': 7.501089953620976e-07},
 'skewness': [-0.006285175681114197],
 'stddev': [2.820275068283081],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'histograms': [],
 'mean': [0.2963513135910034,
  0.49683526158332825,
  1.091741681098938,
  2.0924882888793945],
 'names': ['BrownianMotion'],
 'npaths': [1000000, 1000000, 1000000, 1000000],
 'run_time_seconds': {'mean': 1.945744514465332,
  'n': 512,
  'skewness': None,
  'stddev': None},
 'skewness': [-0.0006850513746030629,
  0.001324846176430583,
  -0.0011668088845908642,
  -0.00040112141869030893],
 'stddev': [1.9999676942825317,
  2.8309290409088135,
  4.477072715759277,
  6.332667827606201],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [100, 200, 500, 1000]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

0.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.305035263299942,
  0.2850147485733032,
  0.5275639891624451,
  0.4439034163951874,
  1.115442156791687,
  1.06523597240448],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.3050948977470398,
  'n': 512,
  'skewness': None,
  'stddev': 6.02680811425671e-05},
 'skewness': [-0.04894294589757919,
  0.018376624211668968,
  -0.015077664516866207,
  0.012991233728826046,
  -0.001292679226025939,
  -0.01008482463657856],
 'stddev': [1.995850682258606,
  1.9956544637680054,
  2.825455665588379,
  2.8237504959106445,
  4.4687323570251465,
  4.488648891448975],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [9]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.295841,0.019953,1.995275,-0.005259
1,BrownianMotion,,1,1.0,2000,10000,0.308946,0.019879,1.987866,-0.010148
2,BrownianMotion,,0,2.0,4000,10000,0.473134,0.028226,2.822643,0.008908
3,BrownianMotion,,1,2.0,4000,10000,0.535114,0.028375,2.837510,-0.012088
4,BrownianMotion,,0,5.0,10000,10000,1.030397,0.044354,4.435421,-0.008756
5,BrownianMotion,,1,5.0,10000,10000,1.211153,0.044689,4.468917,-0.011768
